Note: If the pipeline is taking too long to load, consider allocating a SLURM job via command line with more CPU memory:
```
salloc --gres=gpu:2 --qos=high --partition=t4v2 --mem=64G

```

At a minimum, it is best to allocate at least 32GB of CPU memory to load this model

## Prompting Instruction Fine-tuned language models
Language Models such as OPT and LLaMA are trained to do next-token prediction (autocompletion). As a result, they might easily miss the fact that we expect it to answer our question instead of providing a reasonable (but not very helpful) "autocompletion" to our query. One way to steer the LM away from this behavior is instruction fine-tuning.

In this notebook, you will find examples on how to work with instruction fine-tuned (IFT) LLMs, and how they can help simplify your prompt design workflow. 

Overall steps for calling an IFT model can include:
- Loading the model- either locally as a HuggingFace pipeline, or using a remote model API as in the Kaleidoscope examples;
- Pre-processing your text query. You might need to add special tokens to your input to achieve full instruction fine-tuning potentials; And,
- Extracting structured information from the model output.

To speed things up, we've cached weights of a number of IFT models on the Vector cluster. If you prefer, you may also download these weights directly from the HuggingFace Hub:
- Koala-7B: https://huggingface.co/TheBloke/koala-7B-HF
- Alpaca-7B: https://huggingface.co/chavinlo/alpaca-native

## Koala IFT model
Similar to Alpaca, the [Koala](https://bair.berkeley.edu/blog/2023/04/03/koala/) model is an instruction fine-tuned version of Facebook AI's LLaMA LLM. 

The following example is based on the 7B version of Koala (13GB at 16-bit).

Note that initializing the pipeline might take a while- around five minutes for a 7B model. 

In [1]:
import torch
from transformers import pipeline

MODEL_PATH = "/ssd005/projects/llm/koala-7B-HF"
generator = pipeline("text-generation", model=MODEL_PATH) 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Pre-process Query
To build an IFT LLM, we fine-tune the base model (e.g., OPT, LLaMA) on demonstrations where the LLM needs to produce outputs that follow human instructions. To remind the model about its role as the "assistant" and not the "human", we would add special separator tokens between the human instruction and what the model needs to generate. 

For example, in the Koala model, the authors added the following separators between instructions and demonstrations:

- Add "BEGINNING OF CONVERSATION: " to the beginning of each conversation,
- Add "USER: " before each human input,
- Add "GPT: " after the human query, and
- Add "\</s\>" (a special token) to the end of each LM output.

During inference, we would need to add the same set of tokens to make the most out of the instruction fine-tuned model. 

Refer to Koala's [documentation](https://github.com/young-geng/EasyLM/blob/main/docs/koala.md) for more details.

Google's [blog post](https://ai.googleblog.com/2021/10/introducing-flan-more-generalizable.html) on FLAN provides additional detail on how Instruction Fine-Tuning can potentially improve the zero-shot performance of LLMs.

In [2]:
def preprocess_koala_input(user_input: str) -> str:
    return "BEGINNING OF CONVERSATION: USER: " + user_input + " GPT:"

### Run pre-processed query through the pipeline

In [3]:
query = """We had a great experience at the restaurant, food was delicious, but the service was kinda bad.             

What is the sentiment on the restaurant, positive or negative? Explain your reasoning."""  # noqa: W291

text_input = preprocess_koala_input(query)

In [8]:
print("Input to model: \n###\n" + text_input + "\n###")

hf_pipeline_output = generator(text_input, max_new_tokens=256)

print("Pipeline output: \n###\n" + str(hf_pipeline_output) + "\n###")

Input to model: 
###
BEGINNING OF CONVERSATION: USER: We had a great experience at the restaurant, food was delicious, but the service was kinda bad.             

What is the sentiment on the restaurant, positive or negative? Explain your reasoning. GPT:
###


KeyboardInterrupt: 

### Extract useful info from model output
There are many ways to extract structural information (e.g., binary label for sentiment: positive/label) from the natural text output of the language model. 

Since the focus of this notebook is on prompting instruction fine-tuned models, we will demonstrate only a basic example where we delete previous input to the model and keep only the newly-generated tokens.

In [6]:
def extract_koala_response(model_output: str, previous_input: str) -> str:
    return model_output.replace(previous_input, "")

In [ ]:
model_output = hf_pipeline_output[0]["generated_text"]
extracted_output = extract_koala_response(model_output, previous_input=text_input)

print("Extracted output: \n###\n" + extracted_output + "\n###")